In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

actual = pd.read_csv('Italy_data_actual.csv')
actual.head()

,timestamps,prices,consumption
0,2018-01-01 00:00:00,45.82,22.98
1,2018-01-01 01:00:00,44.16,21.81
2,2018-01-01 02:00:00,42.24,20.76
3,2018-01-01 03:00:00,39.29,19.97
4,2018-01-01 04:00:00,36.00,19.49


In [2]:
# just segregate actuals for date Dec 1, 2019 to Dec 7, 2019
actual['timestamps'] = pd.to_datetime(actual['timestamps'])
actual = actual[(actual['timestamps'] >= datetime(2019, 12, 1)) & (actual['timestamps'] < datetime(2019, 12, 8))]

forecast_prices = pd.read_csv("Italy_Time_LLM_prices.csv")
forecast_demand = pd.read_csv("Italy_Time_LLM_consumption.csv")

# Ensure all data is in the same format
actual['timestamps'] = pd.to_datetime(actual['timestamps'])
forecast_prices['timestamps'] = pd.to_datetime(forecast_prices['timestamps'])
forecast_demand['timestamps'] = pd.to_datetime(forecast_demand['timestamps'])

actual.sort_values('timestamps', inplace=True)
forecast_prices.sort_values('timestamps', inplace=True)
forecast_demand.sort_values('timestamps', inplace=True)



In [3]:
forecast_prices.reset_index(drop=True, inplace=True)
forecast_demand.reset_index(drop=True, inplace=True)

#drop nan values
forecast_prices.dropna(inplace=True)
forecast_demand.dropna(inplace=True)

forecast_prices.head(), forecast_prices.tail()

(           timestamps  prices  TimeLLM_pred
 0 2019-01-01 00:00:00   51.00     45.970184
 1 2019-01-01 01:00:00   46.27     45.739730
 2 2019-01-01 02:00:00   39.78     44.223320
 3 2019-01-01 03:00:00   27.87     42.651825
 4 2019-01-01 04:00:00   22.00     41.655990,
               timestamps  prices  TimeLLM_pred
 8754 2019-12-31 19:00:00   56.48     50.095870
 8755 2019-12-31 20:00:00   53.24     47.453490
 8756 2019-12-31 21:00:00   50.68     44.166523
 8757 2019-12-31 22:00:00   48.61     38.488270
 8758 2019-12-31 23:00:00   42.30     34.634598)

In [4]:
forecast_prices.tail()

,timestamps,prices,TimeLLM_pred
8754,2019-12-31 19:00:00,56.48,50.095870
8755,2019-12-31 20:00:00,53.24,47.453490
8756,2019-12-31 21:00:00,50.68,44.166523
8757,2019-12-31 22:00:00,48.61,38.488270
8758,2019-12-31 23:00:00,42.30,34.634598


In [5]:
actual.head()

,timestamps,prices,consumption
16774,2019-12-01 00:00:00,65.10,28.68
16775,2019-12-01 01:00:00,62.12,27.12
16776,2019-12-01 02:00:00,59.05,25.99
16777,2019-12-01 03:00:00,59.00,25.62
16778,2019-12-01 04:00:00,56.63,25.66


In [6]:
data_actual =  pd.merge(forecast_prices[['timestamps', 'prices']], forecast_demand[['timestamps', 'consumption']], on='timestamps', how='inner')
# data_actual.rename(columns={'actual_pred_price': 'prices', 'actual_pred_demand': 'consumption'}, inplace=True)
data_actual.sort_values('timestamps', inplace=True)
data_actual.dropna(inplace=True)
# find out the duplicate timestamps in the data_actual dataframe
duplicates = data_actual[data_actual.duplicated('timestamps', keep=False)]

# remove these duplicates keeping the first occurrence
data_actual = data_actual.drop_duplicates('timestamps', keep='first')
data_actual.reset_index(drop=True, inplace=True)
data_actual

,timestamps,prices,consumption
0,2019-01-01 00:00:00,51.00,22.74
1,2019-01-01 01:00:00,46.27,21.50
2,2019-01-01 02:00:00,39.78,20.50
3,2019-01-01 03:00:00,27.87,19.64
4,2019-01-01 04:00:00,22.00,19.30
...,...,...,...
8754,2019-12-31 19:00:00,56.48,34.89
8755,2019-12-31 20:00:00,53.24,32.54
8756,2019-12-31 21:00:00,50.68,30.03
8757,2019-12-31 22:00:00,48.61,27.64


In [9]:
#just help me merge the prices and consumption into one dataframe for the forecasts just likw he actual dataframe
data_forecast =  pd.merge(forecast_prices[['timestamps', 'TimeLLM_pred']], forecast_demand[['timestamps', 'TimeLLM_pred']], on='timestamps', how='inner')
data_forecast.rename(columns={'TimeLLM_pred_x': 'prices', 'TimeLLM_pred_y': 'consumption'}, inplace=True)
data_forecast.sort_values('timestamps', inplace=True)
data_forecast.dropna(inplace=True)
data_forecast.reset_index(drop=True, inplace=True)
data_forecast

,timestamps,prices,consumption
0,2019-01-01 00:00:00,45.970184,23.921310
1,2019-01-01 01:00:00,45.739730,22.560665
2,2019-01-01 02:00:00,44.223320,21.964160
3,2019-01-01 03:00:00,42.651825,21.188221
4,2019-01-01 04:00:00,41.655990,19.946920
...,...,...,...
8754,2019-12-31 19:00:00,50.095870,31.106040
8755,2019-12-31 20:00:00,47.453490,30.085402
8756,2019-12-31 21:00:00,44.166523,28.490654
8757,2019-12-31 22:00:00,38.488270,26.414190


In [10]:
data_forecast.to_csv('Italy_data_forecast_TLLM.csv', index=False)

In [6]:
data_noise =  pd.merge(forecast_prices[['timestamps', 'prices + noise']], forecast_demand[['timestamps', 'consumption + noise']], on='timestamps', how='inner')
data_noise.rename(columns={'prices + noise': 'prices', 'consumption + noise': 'consumption'}, inplace=True)
data_noise.sort_values('timestamps', inplace=True)
data_noise.dropna(inplace=True)
# find out the duplicate timestamps in the data_noise dataframe
duplicates = data_noise[data_noise.duplicated('timestamps', keep=False)]

# remove these duplicates keeping the first occurrence
data_noise = data_noise.drop_duplicates('timestamps', keep='first')
data_noise.reset_index(drop=True, inplace=True)
data_noise

,timestamps,prices,consumption
0,2018-01-01 00:00:00,46.073502,24.515765
1,2018-01-01 01:00:00,43.730295,22.733190
2,2018-01-01 02:00:00,42.851569,19.517337
3,2018-01-01 03:00:00,37.710871,20.132493
4,2018-01-01 04:00:00,36.055618,19.239896
...,...,...,...
17513,2019-12-31 19:00:00,54.310326,33.258347
17514,2019-12-31 20:00:00,52.782918,32.851411
17515,2019-12-31 21:00:00,48.470322,29.049828
17516,2019-12-31 22:00:00,48.921003,28.091693


In [7]:
data_LSTM =  pd.merge(forecast_prices[['timestamps', 'LSTM_pred']], forecast_demand[['timestamps', 'LSTM_pred']], on='timestamps', suffixes=('_price', '_demand'))
data_LSTM.rename(columns={'LSTM_pred_price': 'prices', 'LSTM_pred_demand': 'consumption'}, inplace=True)
data_LSTM.sort_values('timestamps', inplace=True)
data_LSTM.dropna(inplace=True)
# find out the duplicate timestamps in the data_LSTM dataframe
duplicates = data_LSTM[data_LSTM.duplicated('timestamps', keep=False)]

# remove these duplicates keeping the first occurrence
data_LSTM = data_LSTM.drop_duplicates('timestamps', keep='first')
data_LSTM.reset_index(drop=True, inplace=True)
data_LSTM

,timestamps,prices,consumption
0,2018-01-01 00:00:00,76.913203,35.051603
1,2018-01-01 01:00:00,76.245376,36.221259
2,2018-01-01 02:00:00,73.325385,35.518524
3,2018-01-01 03:00:00,69.120745,33.928986
4,2018-01-01 04:00:00,63.816008,32.341698
...,...,...,...
17513,2019-12-31 19:00:00,82.553541,45.659468
17514,2019-12-31 20:00:00,79.519272,44.297204
17515,2019-12-31 21:00:00,77.329024,41.369403
17516,2019-12-31 22:00:00,72.623072,38.391652


In [8]:
data_rf =  pd.merge(forecast_prices[['timestamps', 'RF_pred']], forecast_demand[['timestamps', 'RF_pred']], on='timestamps', suffixes=('_price', '_demand'))
data_rf.rename(columns={'RF_pred_price': 'prices', 'RF_pred_demand': 'consumption'}, inplace=True)
data_rf.sort_values('timestamps', inplace=True)
data_rf.dropna(inplace=True)
# find out the duplicate timestamps in the data_rf dataframe
duplicates = data_rf[data_rf.duplicated('timestamps', keep=False)]

# remove these duplicates keeping the first occurrence
data_rf = data_rf.drop_duplicates('timestamps', keep='first')
data_rf.reset_index(drop=True, inplace=True)
data_rf

,timestamps,prices,consumption
0,2018-01-01 00:00:00,47.318961,24.956646
1,2018-01-01 01:00:00,41.784323,23.929971
2,2018-01-01 02:00:00,37.492516,22.429782
3,2018-01-01 03:00:00,36.728515,21.715821
4,2018-01-01 04:00:00,36.248728,21.876235
...,...,...,...
17513,2019-12-31 19:00:00,74.521129,44.246491
17514,2019-12-31 20:00:00,73.560176,40.480962
17515,2019-12-31 21:00:00,69.016322,37.848983
17516,2019-12-31 22:00:00,65.635389,34.336359


In [9]:
data_actual.to_csv('Italy_data_actual.csv', index=False)
data_noise.to_csv('Italy_data_forecast_NOISE.csv', index=False)
data_LSTM.to_csv('Italy_data_forecast_LSTM.csv', index=False)
data_rf.to_csv('Italy_data_forecast_RF.csv', index=False)